#### Documentation : https://api.mongodb.com/python/current/tutorial.html  https://docs.mongodb.com/v3.0/reference/method/db.dropDatabase/

## 寫進mongoDB

In [2]:
import pymongo
from pymongo import MongoClient
# client = MongoClient('mongodb://doraemon.iis.sinica.edu.tw:27017') #Connect (MongoDB URI format)
# db = client['AntiNuke'] #Getting Database
# collect = db['FBcollection'] #Getting Collection

In [3]:
client = MongoClient()
# client = MongoClient('mongodb://localhost:27017/')  #連到本機

In [4]:
from pymongo import Connection
connection = Connection()#這是在自己的本機建的，所以用connection
# db = connection['Apple'] #在本機建立新的database
# db = client['Apple'] #在伺服器建立新的database
# connection.drop_database('Facebook') #可以移除database

In [5]:
db = connection['Candidate']
# collection = db['comments'] #comments/posts/reactions

In [28]:
# db.pick_homo.drop() #Remove a collection

In [6]:
db.collection_names(include_system_collections=False) #listing all of the collections in our database

[u'comments', u'posts', u'reactions', u'detector_one_by_one', u'user']

## 處理 comments區域 --> 存入叫做"comments"的 collection

In [7]:
#讀檔，把comment跟reaction分開
import json
from os import listdir
from os.path import isfile, join

facebook_id = '46251501064' #Tsai
facebook_page_name = 'Tsai'
# facebook_id = '136845026417486' #Ke
# facebook_page_name = 'Ke'
# facebook_id = '188311137478' #ChinaTimes 
# facebook_page_name = 'ChinaTimes'
# facebook_id = '241284961029'
# facebook_page_name = 'udn' 
# facebook_id = '394896373929368' 
# facebook_page_name = 'ltn' 
# facebook_id = '232633627068' 
# facebook_page_name = 'AppleDaily'

file_path = 'data'+'/'+facebook_id+'/'
onlyfiles = [f for f in listdir(file_path) if isfile(join(file_path, f))]

## 把 第一層回覆 與 reaction以後的分開放
comment_list, reac_list = [],[]

for f in onlyfiles:
    if f.endswith('.json'):
        if f.startswith('comment'):
            comment_list.append(f)
        if f.startswith('reaction'):
            reac_list.append(f)
print 'comment count:', len(comment_list)
# print comment_list[0]
print 'reaction count:', len(reac_list)

comment count: 2229
reaction count: 2407


In [8]:
#檢查有沒有重複抓取的comment跟reaction，如果有的話要刪掉
comment_id={}
for c in comment_list:
    if not comment_id.has_key(c.split('-')[1].split('.')[0]):
        comment_id[c.split('-')[1].split('.')[0]]=0
    comment_id[c.split('-')[1].split('.')[0]]+=1
for co in comment_id:
    if comment_id[co]>2:
        print co

In [9]:
#把 posts的資料load進來
p_file_path = 'data'+'/'+facebook_id+'/'+'posts.json'

import operator
posts = []
with open(p_file_path) as ff:
    for line in ff:
        posts.append(json.loads(line))

# for v, c in enumerate(posts[0]):
#     print v
print len(posts[0])

2412


In [10]:
### 第一層回覆的檔案
import operator

first_c_data = []
comment_id = [] #post_id
for h, j in enumerate(comment_list):
#     print comment_list[h] # 檔名（為貼文的id）
    comment_id.append(comment_list[h].split('-')[1].split('.')[0])
    
    with open(file_path+comment_list[h]) as fff:
        for line in fff:
            first_c_data.append(json.loads(line))
# print comment_id 
# print first_c_data[1]

### 寫進叫做 “comments"的collection

In [51]:
import datetime
# collection名＋.find() 可以印出在一個collection中的所有內容 --> 不要印，很恐怖～

for cc in first_c_data:
    for c in cc: 
        pt = datetime.datetime.strptime(c['created_time'], "%Y-%m-%dT%H:%M:%S+%f")
        t = pt.timetuple()  #是一個神奇的函式
        y = c['created_time'][0:4][-2:]
        k = c['id'].split('_')[0]
        for o in posts[0]:
            if k == o['id'].split('_')[1]:
#                 print o['id']
                post_time = datetime.datetime.strptime(o['created_time'], "%Y-%m-%dT%H:%M:%S+%f")
                pp = post_time.timetuple()

    #     print t[0] #year
    #     print t[1] #month
    #     print t[2] #date
#         print t[3] #hour
    #     print t[4] #min
    #     print t[5] #sec
    #     print pt_3, pt_2

    # for p in collect.find()[77:78]:
    #     print p

                db.comments.insert({'comment':c['message'],
                                'post_time':str(pp[1])+'/'+str(pp[2])+'/'+str(y)+' '+str(pp[3])+':'+str(pp[4]),
                                'user_id':c['from']['id'],
                                'time':str(t[3])+':'+str(t[4])+':'+str(t[5]),
                                'user_name':c['from']['name'],
                                'comment_id':c['id'],
                                'month':t[1],
                                'post_id':c['id'].split('_')[0],
                                'page_name':facebook_page_name,
                                'year':t[0],
                                'date':str(t[1])+'/'+str(t[2])+'/'+str(y)+' '+str(t[3])+':'+str(t[4]),
                                'page_id':facebook_id,
                                'day':t[2],
                                'comment_count':c['comment_count'],
                                'like_count':c['like_count']})
               
db.comments.count()
db.update
# comments.update()
print db.comments.count()

1232243


## 處理post 區域 --> 存入叫做 FBcollection的collection

In [17]:
import datetime
# collection名＋.find() 可以印出在一個collection中的所有內容 --> 不要印，很恐怖～
    
for v, b in enumerate(posts[0]): 
    pt = datetime.datetime.strptime(b['created_time'], "%Y-%m-%dT%H:%M:%S+%f")
    t = pt.timetuple()  #是一個神奇的函式
    
    m = ''
    try:
        m = b['message']

    #     print t[0] #year
    #     print t[1] #month
    #     print t[2] #date
    #     print t[3] #hour
    #     print t[4] #min
    #     print t[5] #sec

        db.posts.insert({'Created_Time':str(t[1])+'/'+str(t[2])+'/'+str(t[0])+' '+str(t[3])+':'+str(t[4]),
                        'Page_ID':facebook_id,
                        'Message':m,
                        'Page_Name':facebook_page_name,
                        'Year':t[0],
                        'Day':t[2],
                        'URL':b['permalink_url'],
                        'Time':str(t[3])+':'+str(t[4])+':'+str(t[5]),
                        'Month':t[1],
                        'Post_ID':b['id']})
    except:
        m = ''
        db.posts.insert({'Created_Time':str(t[1])+'/'+str(t[2])+'/'+str(t[0])+' '+str(t[3])+':'+str(t[4]),
                        'Page_ID':facebook_id,
                        'Message':m,
                        'Page_Name':facebook_page_name,
                        'Year':t[0],
                        'Day':t[2],
                        'URL':b['permalink_url'],
                        'Time':str(t[3])+':'+str(t[4])+':'+str(t[5]),
                        'Month':t[1],
                        'Post_ID':b['id']})
#         print 'post_id', c['id']
# db.posts.find()
db.update
db.posts.count()
print db.posts.count()

3502


## 處理reaction 區域
1. 按照不同的reactions存
2. 存入叫做 reactions的collection
3. 如果沒有任何一種reaction的話就不存

In [53]:
#load reactions.json
# reac_list --> reactions data inside
import operator
reac_data = {}
for h, j in enumerate(reac_list):
#     print reac_list[h] # 檔名（為貼文的id）
    reac_id = reac_list[h].split('-')[1].split('.')[0]
    
    with open(file_path+reac_list[h]) as fff:
        for line in fff:
            reac_data[reac_id] = json.loads(line)
# print reac_data.keys()

for rr,rv in reac_data.items():
    LIKE,SAD,ANGRY,WOW,HAHA,LOVE = [],[],[],[],[],[]
    rdata = []
    if len(rv) == 1:  #如果reaction只有一串（有時候量太大會有一串以上）
        if len(rv[0]) == 1: # [[""]]  #如果reactions中什麼都沒有，就略過
            pass
        else:
            if "paging" in rv[0]:
                rdata = rv[0]['data']  #有些reactions中有"paging",那資料是rv[0]['data']
            else:
                rdata = rv[0]  #如果沒有"paging",那資料就是rv[0]
            
    else:
        for rd in range(len(rv)): #如果是一串以上
            try: 
                rdata.append(rv[rd]['data'][0])  #試著把資料中所有的'data'[0]加進去
            except:
                try:
                    if len(rv[rd][0])==1:  #如果裡面是空的
                        print rv[rd]
                        continue
                    else:
                        rdata = rv[rd][0] 
                except:
                    print rv[rd]
                        
#     fname = file_path + '000-%s.json'%(rr)
#     with open(fname, 'w') as fout:
#         json.dump(rdata, fout)
    
    for r in range(len(rdata)):
        try:
    #     print rdata[r]['type'],rdata[r]['id'] #LIKE 1339671589393499
            if rdata[r]['type'] == 'LIKE':
                LIKE.append(rdata[r]['id'])
            if rdata[r]['type'] == 'SAD':
                SAD.append(rdata[r]['id'])
            if rdata[r]['type'] == 'ANGRY':
                ANGRY.append(rdata[r]['id'])
            if rdata[r]['type'] == 'WOW':
                WOW.append(rdata[r]['id'])
            if rdata[r]['type'] == 'HAHA':
                HAHA.append(rdata[r]['id'])
            if rdata[r]['type'] == 'LOVE':
                LOVE.append(rdata[r]['id'])
        except:
#             print rr
            pass
    ###取一些post的資料加進去###
    p_link = []
    for o in posts[0]:
        if rr == o['id']:
            p_link = o['permalink_url']
###寫入資料庫###
    db.reactions.insert({'LIKE':LIKE,
                    'SAD':SAD,
                    'ANGRY':ANGRY,
                    'WOW':WOW,
                    'HAHA':HAHA,
                    'LOVE':LOVE,
                    'Page_ID':facebook_id,
                    'Page_Name':facebook_page_name,
                    'Post_ID':rr,
                    'Link':p_link})
db.reactions.count()
db.update
# comments.update()
print db.reactions.count()

2463
